In [1]:
import pandas as pd
import numpy as np
import cv2

from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval

import torch.cuda
import torch.nn as nn

from sklearn.model_selection import train_test_split

In [65]:
import Utils.PTModel.Losses as Losses
import Utils.PTModel.Models as Models
import Utils.Data.DataLoading as DataLoading

In [67]:
crossValidationData = DataLoading.LoadCrossValTrainTestData("../data/tomData/unshuffled3x3tiles.csv", shuffle=True)

{'wall', 'element', 'pipe', 'passable', 'block', 'solid', 'moving', 'openable', 'hazard', 'climbable', 'collectable', 'empty', 'breakable'}
tensor([3.6799, 3.9129, 5.4777, 1.9535, 3.4377, 1.5683, 5.0192, 4.7846, 4.4612,
        4.9633, 5.0192, 2.0802, 4.4744])


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/Data/DataLoading.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([0.0000, 0.0000, 6.8108, 1.5074, 0.0000, 1.9175, 6.3522, 7.5223, 4.8912,
        2.5969, 3.9553, 2.0638, 2.4590])


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/Data/DataLoading.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0217, 5.2547, 6.8195, 1.5749, 4.7795, 1.8468, 6.6372, 6.1263, 5.0308,
        2.6287, 4.0014, 2.1699, 2.4773])


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/Data/DataLoading.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0529, 5.2859, 6.8507, 1.5881, 4.8106, 1.8553, 6.8327, 6.4272, 4.9445,
        2.6360, 3.9953, 2.1532, 2.4989])


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/Data/DataLoading.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0540, 5.2870, 0.0000, 1.5883, 4.8118, 1.8340, 6.8889, 6.1587, 4.9871,
        2.6372, 4.0512, 2.1702, 2.5257])


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/srcTom/Utils/Data/DataLoading.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


In [56]:
def TextTileToImage(tileArray, tileSize, spritePath, savePath=None):

    outputImage = np.empty((tileSize*tileArray.shape[0], tileSize*tileArray.shape[1], 3), dtype=np.uint8)

    fileName = ""

    for i, row in enumerate(tileArray):
        for j, tile in enumerate(row):

            tile = '@' if tile == '.' else tile
            fileName += tile

            tileImage = cv2.cvtColor(cv2.imread(f"{spritePath}/{tile}.png"), cv2.COLOR_BGR2RGB)
            outputImage[i*tileSize:(i+1)*tileSize, j*tileSize:(j+1)*tileSize] = tileImage

    if savePath:
        cv2.imwrite(f"{savePath}/{fileName}.png", outputImage)

    return outputImage

def TFIDFWeightVector(data, uniqueClasses):
    # Initialize TfidfVectorizer from sklearn
    vectorizer = TfidfVectorizer(stop_words=None)

    # Fit the TfidfVectorizer to affordance values in the training set
    vectorizer.fit_transform(data["affordances"].apply(lambda x: str(x)))

    # Add the weights created for each affordance class to a easily indexable dictionary
    newDict = {affordanceClass: vectorizer.idf_[vectorizer.vocabulary_[affordanceClass]] if affordanceClass in vectorizer.vocabulary_ else 0.0 for affordanceClass in uniqueClasses}

    # Average each weight and scale by 1000
    # weightFreq = {k: v / sum(newDict.values()) for k, v in newDict.items()}
    # weightVector = [v * 1000 for v in newDict.values()] Forked github code dont know why * 1000??
    weightVector = [v for v in newDict.values()]

    tensorFromList = torch.tensor(weightVector, dtype=torch.float32)
    print(tensorFromList)

    return newDict, tensorFromList

In [54]:
def LoadTrainTestData(pathToDataCsv, testSetSize=0.1, shuffle=False, randomState=1):

    dataFrame = pd.read_csv(pathToDataCsv)

    dataFrame['affordances'] = dataFrame['affordances'].apply(lambda x: literal_eval(str(x)))
    dataFrame['tiles'] = dataFrame['tiles'].apply(lambda x: np.array(literal_eval(str(x))))

    return train_test_split(dataFrame, test_size=testSetSize, random_state=randomState, shuffle=shuffle)

def LoadCrossValTrainTestData(pathToDataCsv, shuffle=False, randomState=1):

    crossValDataDict = {}

    spritePaths = {
        "kidicarus": "../data/tomData/sprites/kidicarus", 
        "loderunner": "../data/tomData/sprites/loderunner",
        "megaman": "../data/tomData/sprites/megaman",
        "supermariobros": "../data/tomData/sprites/supermariobros",
        "thelegendofzelda": "../data/tomData/sprites/thelegendofzelda",
    }

    dataFrame = pd.read_csv(pathToDataCsv)

    dataFrame['affordances'] = dataFrame['affordances'].apply(lambda x: literal_eval(str(x)))
    dataFrame['tiles'] = dataFrame['tiles'].apply(lambda x: np.array(literal_eval(str(x))))

    dataFrame['image'] = [TextTileToImage(row['tiles'], 16, spritePaths[row['gamename']]) for index, row in dataFrame.iterrows()]

    uniqueAffordances = set([affordance for affordanceList in dataFrame['affordances'] for affordance in affordanceList])

    print(uniqueAffordances)

    if shuffle:
        dataFrame = dataFrame.sample(frac=1, random_state=randomState).reset_index()

    for gameName in dataFrame['gamename'].unique():
        testData = dataFrame[dataFrame['gamename'] == gameName]
        trainData = dataFrame.drop(testData.index)

        affordanceDict, tfidfWeightArray = TFIDFWeightVector(trainData, uniqueAffordances)

        trainData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in trainData.iterrows()]
        testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]
        
        crossValDataDict[gameName] = {"trainData": trainData, "testData": testData, "weightArray": tfidfWeightArray}
    
    return crossValDataDict

In [57]:
# Loading standard train test split
crossValidationData = LoadCrossValTrainTestData("../data/tomData/unshuffled3x3tiles.csv", shuffle=True)

{'wall', 'element', 'pipe', 'passable', 'block', 'solid', 'moving', 'openable', 'hazard', 'climbable', 'collectable', 'empty', 'breakable'}
tensor([3.6799, 3.9129, 5.4777, 1.9535, 3.4377, 1.5683, 5.0192, 4.7846, 4.4612,
        4.9633, 5.0192, 2.0802, 4.4744])


/tmp/ipykernel_40637/211859064.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([0.0000, 0.0000, 6.8108, 1.5074, 0.0000, 1.9175, 6.3522, 7.5223, 4.8912,
        2.5969, 3.9553, 2.0638, 2.4590])


/tmp/ipykernel_40637/211859064.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0217, 5.2547, 6.8195, 1.5749, 4.7795, 1.8468, 6.6372, 6.1263, 5.0308,
        2.6287, 4.0014, 2.1699, 2.4773])


/tmp/ipykernel_40637/211859064.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0529, 5.2859, 6.8507, 1.5881, 4.8106, 1.8553, 6.8327, 6.4272, 4.9445,
        2.6360, 3.9953, 2.1532, 2.4989])


/tmp/ipykernel_40637/211859064.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


tensor([5.0540, 5.2870, 0.0000, 1.5883, 4.8118, 1.8340, 6.8889, 6.1587, 4.9871,
        2.6372, 4.0512, 2.1702, 2.5257])


/tmp/ipykernel_40637/211859064.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testData["encodedAffordances"] = [np.sum(np.array([np.where(np.array(list(affordanceDict.keys())) == affordance, 1, 0) for affordance in row['affordances']]), axis=0) for index, row in testData.iterrows()]


In [24]:
class WeightedBCE(nn.Module):

    def __init__(self, weightedArray, debug=False):
        super().__init__()

        self.debug = debug

        self.weighedArray = weightedArray

    def forward(self, yPred, yTrue):

        bce_array = nn.functional.binary_cross_entropy(yPred, yTrue, reduction="none")
        weighted_array = torch.mul(bce_array, self.weighedArray)

        if self.debug:
            print(weighted_array.shape)

        bce_sum = torch.sum(weighted_array, axis=1)
        loss = torch.div(bce_sum, 13.0)
        loss = torch.mean(loss)

        return loss

In [40]:
class TileEmbeddingVAE(nn.Module):

    def __init__(self, debug=False):
        super().__init__()

        self.debug = debug

        self.imageEncoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=3),
            nn.BatchNorm2d(32),
            nn.Tanh(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.Tanh(),
            nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.Tanh(),
            nn.Flatten(),
        )

        self.textEncoder = nn.Sequential(
            nn.Linear(13, 32),
            nn.Tanh(),
            nn.Linear(32, 16),
            nn.Tanh(),
        )

        self.embeddingLayer = nn.Linear(4112, 256)

        self.imageDecoder = nn.Sequential(
            nn.Linear(256, 4096),
            nn.Unflatten(1, (16, 16, 16)),
            nn.ConvTranspose2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.Tanh(),
            nn.ConvTranspose2d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.Tanh(),
            nn.ConvTranspose2d(32, 3, kernel_size=3, stride=1, padding=1),
        )

        self.textDecoder = nn.Sequential(
            nn.Linear(256, 16),
            nn.Tanh(),
            nn.Linear(16, 32),
            nn.Tanh(),
            nn.Linear(32, 13),
            nn.Sigmoid(),
        )

    def encode(self, xImages, xText):
        encodedImage = self.imageEncoder(xImages)

        if self.debug:
            print(f"xText shape: {xText.shape}")

        encodedText = self.textEncoder(xText)

        if self.debug:
            print("EncodedImage shape: ", encodedImage.shape)
            print("encodedText shape: ", encodedText.shape)

        concatenateEmbeddding = torch.cat((encodedImage, encodedText), 1)

        embedding = self.embeddingLayer(concatenateEmbeddding)

        return embedding

    def decode(self, embedding):
        decodedImage = self.imageDecoder(embedding)
        decodedText = self.textDecoder(embedding)
        return decodedImage, decodedText

    def forward(self, xImages, xText):
        # Encoder
        encodedEmbedding = self.encode(xImages, xText)

        # Decoder
        yPredImage, yPredText = self.decode(encodedEmbedding)
        
        return yPredImage, yPredText

In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def TrainModel(dataObj, epochs, batchSize):
    # dataObj = {"trainData": trainingdata, "testData": testData, "weightArray": tfidfWeightArray}

    model = TileEmbeddingVAE()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    imageCritierion = nn.MSELoss()
    textCritierion = WeightedBCE(dataObj["weightArray"].to(device))

    imageLossWeight = 0.8
    textLossWeight = 1.0 - imageLossWeight

    model.to(device)
    model.train()

    trainData = dataObj["trainData"]
    losses = []

    for i in range(epochs):

        losses.append([])

        for j in range(0, trainData.shape[0], batchSize):

            xImages = np.array(trainData.iloc[j:j+batchSize]["image"].tolist())
            yImages = xImages[:, 16:32, 16:32, :]
            # print(xImages.shape)
            # print(yImages.shape)

            xImageBatch = torch.tensor(xImages, dtype=torch.float32)
            xImageBatch = xImageBatch.reshape((-1, 3, 48, 48))
            xImageBatch = xImageBatch.to(device)

            yImageBatch = torch.tensor(yImages, dtype=torch.float32)
            yImageBatch = yImageBatch.reshape((-1, 3, 16, 16))
            yImageBatch = yImageBatch.to(device)
            
            xTextbatch = torch.tensor(trainData.iloc[j:j+batchSize]["encodedAffordances"].tolist(), dtype=torch.float32).to(device)

            yPredImages, yPredTexts = model(xImageBatch, xTextbatch)
            # print(yPredImages.shape)
            # print(yImageBatch.shape)

            imageLoss = imageCritierion(yPredImages, yImageBatch)
            textLoss = textCritierion(yPredTexts, xTextbatch)
            # print(imageLoss)
            # print(textLoss)

            loss = torch.add(torch.mul(imageLoss, imageLossWeight), torch.mul(textLoss, textLossWeight))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses[i].append(loss.cpu().detach().item())

        print(f"Epoch {i}: loss {sum(losses[i])/len(losses[i])}")

In [35]:
crossValidationData["kidicarus"]["weightArray"]

tensor([4.8106, 2.4989, 2.6360, 3.9953, 5.2859, 2.1532, 4.9445, 6.8327, 6.4272,
        1.5881, 6.8507, 1.8553, 5.0529])

In [36]:
TrainModel(crossValidationData["kidicarus"], 10, 32)

Epoch 0: loss 6198.500048174906
Epoch 1: loss 3306.449803176931
Epoch 2: loss 2142.480155306915
Epoch 3: loss 1385.9383625553603
Epoch 4: loss 901.5092200339838
Epoch 5: loss 578.3434430891056
Epoch 6: loss 356.36727619490097
Epoch 7: loss 209.0026473361114
Epoch 8: loss 119.64031777653008
Epoch 9: loss 66.00241887848513


In [68]:
for k, v in crossValidationData.items():
    print(k)
    TrainModel(v, 1, 32)
    print("Done")

loderunner
Epoch 0: loss 9428.914293791118
Done
thelegendofzelda
Epoch 0: loss 6191.405737900152
Done
megaman
Epoch 0: loss 5967.292700574806
Done
kidicarus
Epoch 0: loss 6195.930823692908
Done
supermariobros
Epoch 0: loss 6297.045042720527
Done
